In [ ]:
# ---------- FULL ONE-CELL SCRIPT FOR GOOGLE COLAB ----------
# Run the whole cell in Colab. It will:
# 1) install packages
# 2) write streamlit_dashboard.py
# 3) start ngrok and run Streamlit, printing a public URL to open.

# 1) Install packages
!pip install -q streamlit pandas plotly openpyxl numpy pyngrok

# 2) Write the Streamlit app file (streamlit_dashboard.py)
dashboard_code = r'''
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
from io import StringIO

st.set_page_config(page_title="Interactive Business Dashboard", layout="wide", initial_sidebar_state="expanded")

st.title("📊 Interactive Business Dashboard — Global Superstore")
st.markdown("Upload your **Global Superstore dataset (CSV or Excel)** to explore Sales, Profit, and Segment-wise performance.")

@st.cache_data
def load_data(file) -> pd.DataFrame:
    try:
        if file.name.lower().endswith((".xls", ".xlsx")):
            df = pd.read_excel(file, engine="openpyxl")
        else:
            content = file.read()
            if isinstance(content, (bytes, bytearray)):
                content = content.decode('utf-8', errors='replace')
            df = pd.read_csv(StringIO(content))
    except Exception as e:
        st.error(f"❌ Failed to read file: {e}")
        return pd.DataFrame()
    return df

def clean_prepare(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    df = df.copy()
    df.columns = [c.strip().replace(' ', '_') for c in df.columns]

    # Convert date-like columns
    for c in df.columns:
        if "date" in c.lower():
            df[c] = pd.to_datetime(df[c], errors='coerce')

    # Numeric conversions
    for col in ['Sales', 'Profit', 'Quantity', 'Discount']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    # Map common names to consistent names
    rename_map = {}
    col_map = {
        'Sales': ['Sales', 'sales'],
        'Profit': ['Profit', 'profit'],
        'Region': ['Region', 'region'],
        'Category': ['Category', 'category'],
        'Sub-Category': ['Sub-Category', 'sub_category', 'Sub_Category'],
        'Customer Name': ['Customer Name', 'Customer', 'customer_name', 'customer'],
        'Order Date': ['Order Date', 'Order_Date', 'OrderDate', 'order_date']
    }
    for key, vals in col_map.items():
        for v in vals:
            if v in df.columns:
                rename_map[v] = key
    if rename_map:
        df.rename(columns=rename_map, inplace=True)

    # Fill missing categorical columns
    for c in ['Region', 'Category', 'Sub-Category', 'Customer Name']:
        if c in df.columns:
            df[c] = df[c].fillna("Unknown").astype(str)

    # Ensure Sales and Profit are numeric
    for c in ['Sales', 'Profit']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0.0)

    # Year and Month
    if 'Order Date' in df.columns:
        df['Year'] = df['Order Date'].dt.year
        df['Month'] = df['Order Date'].dt.month
    else:
        df['Year'] = np.nan
        df['Month'] = np.nan

    return df

def compute_kpis(df):
    total_sales = df['Sales'].sum() if 'Sales' in df.columns else 0.0
    total_profit = df['Profit'].sum() if 'Profit' in df.columns else 0.0
    return total_sales, total_profit

# Sidebar: file upload and filters
st.sidebar.header("📂 Upload & Filters")
uploaded_file = st.sidebar.file_uploader("Upload Global Superstore dataset (CSV / Excel)", type=['csv','xlsx','xls'])

if uploaded_file is None:
    st.sidebar.info("No file uploaded yet. You can upload a CSV or Excel file exported from Global Superstore dataset.")
    st.stop()

with st.spinner("Loading data..."):
    raw_df = load_data(uploaded_file)
    if raw_df.empty:
        st.error("Uploaded file couldn't be read or it is empty. Please try another file.")
        st.stop()
    df = clean_prepare(raw_df)

# Filter widgets
regions = ["All"] + sorted(df['Region'].unique()) if 'Region' in df.columns else ["All"]
categories = ["All"] + sorted(df['Category'].unique()) if 'Category' in df.columns else ["All"]
subcats = ["All"] + sorted(df['Sub-Category'].unique()) if 'Sub-Category' in df.columns else ["All"]

selected_regions = st.sidebar.multiselect("Region", options=regions, default=["All"])
selected_categories = st.sidebar.multiselect("Category", options=categories, default=["All"])
selected_subcats = st.sidebar.multiselect("Sub-Category", options=subcats, default=["All"])

# Apply filters
filtered = df.copy()
if 'Region' in filtered.columns and selected_regions and "All" not in selected_regions:
    filtered = filtered[filtered['Region'].isin(selected_regions)]
if 'Category' in filtered.columns and selected_categories and "All" not in selected_categories:
    filtered = filtered[filtered['Category'].isin(selected_categories)]
if 'Sub-Category' in filtered.columns and selected_subcats and "All" not in selected_subcats:
    filtered = filtered[filtered['Sub-Category'].isin(selected_subcats)]

# KPIs row
total_sales, total_profit = compute_kpis(filtered)
col1, col2 = st.columns([2,2])
col1.metric("💰 Total Sales", f"${total_sales:,.2f}")
col2.metric("📈 Total Profit", f"${total_profit:,.2f}")

# Top 5 customers by sales
st.subheader("🏆 Top 5 Customers by Sales")
if 'Customer Name' in filtered.columns and 'Sales' in filtered.columns:
    top_customers = (filtered.groupby('Customer Name')['Sales']
                     .sum()
                     .sort_values(ascending=False)
                     .head(5)
                     .reset_index())
    fig_top = px.bar(top_customers, x='Customer Name', y='Sales', title="Top 5 Customers by Sales", text_auto='.2s')
    st.plotly_chart(fig_top, use_container_width=True)
else:
    st.info("Customer Name or Sales column not found in dataset.")

# Sales & Profit over time
st.subheader("📅 Sales and Profit Over Time")
if 'Order Date' in filtered.columns:
    ts = (filtered.groupby('Order Date')[['Sales','Profit']].sum().reset_index().sort_values('Order Date'))
    if not ts.empty:
        fig_ts = px.line(ts, x='Order Date', y=['Sales','Profit'], labels={'value':'Amount', 'variable':'Metric'}, title="Sales & Profit Over Time")
        st.plotly_chart(fig_ts, use_container_width=True)
    else:
        st.info("No time-series data after grouping by Order Date.")
else:
    st.info("Order Date column not detected; cannot show time series.")

# Category & Sub-Category performance
st.subheader("📦 Category & Sub-Category Performance")
if 'Category' in filtered.columns and 'Sales' in filtered.columns:
    cat_perf = (filtered.groupby('Category')[['Sales','Profit']].sum().reset_index().sort_values('Sales', ascending=False))
    fig_cat = px.bar(cat_perf, x='Category', y=['Sales','Profit'], barmode='group', title="Category Performance (Sales vs Profit)")
    st.plotly_chart(fig_cat, use_container_width=True)
else:
    st.info("Category or Sales column missing.")

# Region performance
st.subheader("🌍 Region Performance")
if 'Region' in filtered.columns and 'Sales' in filtered.columns:
    reg_perf = filtered.groupby('Region')[['Sales','Profit']].sum().reset_index().sort_values('Sales', ascending=False)
    fig_reg = px.bar(reg_perf, x='Region', y='Sales', title="Sales by Region")
    st.plotly_chart(fig_reg, use_container_width=True)
else:
    st.info("Region or Sales column missing.")

# Data table with ability to download filtered dataset
st.subheader("📋 Filtered Data Preview")
st.dataframe(filtered.head(200))

def convert_df_to_csv_bytes(df):
    return df.to_csv(index=False).encode('utf-8')

csv_bytes = convert_df_to_csv_bytes(filtered)
st.download_button("⬇️ Download Filtered Data as CSV", data=csv_bytes, file_name="filtered_global_superstore.csv", mime="text/csv")

st.markdown("---")
st.markdown("**Notes & Tips:**")
st.markdown("- If your dataset uses different column names, the app attempts to detect common variants (e.g., 'Customer', 'Customer Name', 'Sales', 'Profit').")
st.markdown("- For large datasets (>200k rows) the app may be slower; consider sampling or filtering by Year/Region first.")
st.markdown("- Want additional visuals (maps, cohort analysis, custom KPIs)? Ask me and I will extend the app.")
'''
with open("streamlit_dashboard.py", "w", encoding="utf-8") as f:
    f.write(dashboard_code)
print("✅ streamlit_dashboard.py created inside the Colab session.")

!ngrok config add-authtoken 31gleRJuq8LsVqX7iLmKVdlT6R2_5ZnS8nFLiJaTyDQ3dfJ5

# 3) Start ngrok tunnel and run Streamlit
from pyngrok import ngrok
import time

# Kill any old tunnels
try:
    ngrok.kill()
except Exception:
    pass

# Start a new tunnel on port 8501
public_url = ngrok.connect(8501)
print("🌍 Public URL (click to open):", public_url)

# Start Streamlit in the background
get_ipython().system_raw('streamlit run streamlit_dashboard.py --server.port 8501 --server.headless true &')

# Wait a little for the app to boot
time.sleep(5)
print("▶️ Streamlit started in the background. If the page doesn't load immediately, wait 10-20 seconds and refresh the Public URL above.")
# ------------------------------------------------------------


✅ streamlit_dashboard.py created inside the Colab session.
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌍 Public URL (click to open): NgrokTunnel: "https://d1750f9e24cd.ngrok-free.app" -> "http://localhost:8501"
▶️ Streamlit started in the background. If the page doesn't load immediately, wait 10-20 seconds and refresh the Public URL above.
